In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from scipy.signal import find_peaks

Load Data (probably will need to be changed)

In [ ]:

# Load data
def load_spectrum(filename):
    df = pd.read_csv(filename)
    # accept basic column names
    if 'channel' not in df.columns:
        df.columns = ['channel', 'counts']
    return df['channel'].values.astype(int), df['counts'].values.astype(float)

ch_self, n_self = load_spectrum('self_gate.csv')
ch_sca, n_sca   = load_spectrum('sca_gate.csv')

Set SCA-Threshhold: 

In [ ]:

channels = ch_self
# Basic smoothing for peak finding
smooth_sigma = 2.0
n_self_s = gaussian_filter1d(n_self, smooth_sigma)
n_sca_s  = gaussian_filter1d(n_sca, smooth_sigma)

# Compute sensitivity and errors 

with np.errstate(divide='ignore', invalid='ignore'):
    eta = np.where(n_self > 0, n_sca / n_self, np.nan)

# Poisson errors: sigma_N = sqrt(N)
sigma_self = np.sqrt(n_self)
sigma_sca  = np.sqrt(n_sca)

# Propagation for ratio eta = N_sca / N_self
sigma_eta = np.full_like(eta, np.nan)
valid = (n_self > 0) & (n_sca >= 0)
sigma_eta[valid] = eta[valid] * np.sqrt( (sigma_sca[valid]/(n_sca[valid] + 1e-12))**2
                                       + (sigma_self[valid]/(n_self[valid] + 1e-12))**2 )

# Find photopeak region in self-gated spectrum
# Use peak finding to get candidate peak centers (you may need to tweak height, distance)
peaks, _ = find_peaks(n_self_s, height=np.max(n_self_s)*0.05, distance=20)
print("Detected peak channels (candidates):", channels[peaks])

# If the two 60Co peaks are expected, pick the two highest peaks
peak_heights = n_self_s[peaks]
top2_idx = np.argsort(peak_heights)[-2:]
peak_channels = np.sort(channels[peaks][top2_idx])
print("Top two peaks (assumed photopeaks):", peak_channels)

window_width = 200

# Half-height threshold finder 
def half_height_thresholds(chan, ref_counts, gated_counts, peak_channel, window_width=200):
    # 1. define range around peak
    mask = (chan >= peak_channel - window_width) & (chan <= peak_channel + window_width)
    c = chan[mask]; ref = ref_counts[mask]; gate = gated_counts[mask]
    # 2. find the channel where gate reaches half of reference's 'plateau'
    # approximate plateau/reference level as local max near peak
    peak_idx = np.argmax(ref)
    Nref = ref[peak_idx]
    half = 0.5 * Nref
    # find lower threshold: first channel left of peak where gate <= half (crossing)
    left_region = np.where(c <= c[peak_idx])[0]
    right_region = np.where(c >= c[peak_idx])[0]
    # find crossing left-to-right for lower threshold
    lower_candidates = left_region[np.where(gate[left_region] <= half)[0]]
    if lower_candidates.size == 0:
        lower = c[0]
    else:
        lower = c[lower_candidates[-1]]  # nearest to the peak from the left
    # find upper threshold: first channel right of peak where gate <= half
    upper_candidates = right_region[np.where(gate[right_region] <= half)[0]]
    if upper_candidates.size == 0:
        upper = c[-1]
    else:
        upper = c[upper_candidates[0]]
    return int(lower), int(upper), int(Nref)

#  combine both peaks into one window (Option A
lower1, upper1, Nref1 = half_height_thresholds(channels, n_self, n_sca, peak_channels[0], window_width)
lower2, upper2, Nref2 = half_height_thresholds(channels, n_self, n_sca, peak_channels[1], window_width)
print(f"Peak1: lower={lower1}, upper={upper1}, Nref={Nref1}")
print(f"Peak2: lower={lower2}, upper={upper2}, Nref={Nref2}")

#  SCA window covering both peaks
combined_lower = min(lower1, lower2)
combined_upper = max(upper1, upper2)
print(f"Combined SCA window (covers both peaks): [{combined_lower}, {combined_upper}]")


In [ ]:
#  Plots 
plt.figure(figsize=(9,5))
plt.step(channels, n_self, where='mid', label='Self-gated spectrum')
plt.step(channels, n_sca, where='mid', label='SCA-gated spectrum')
plt.axvspan(combined_lower, combined_upper, alpha=0.2, label='SCA window (combined)')
plt.xlabel('Channel'); plt.ylabel('Counts'); plt.legend()
plt.title('MCA Spectra')
plt.savefig('spectra.png', dpi=150)
plt.close()

plt.figure(figsize=(9,5))
plt.errorbar(channels, eta, yerr=sigma_eta, fmt='.', markersize=3, label='Sensitivity')
plt.ylim(-0.1, 1.3)
plt.xlabel('Channel'); plt.ylabel('Sensitivity $\eta$')
plt.title('SCA Sensitivity per Channel')
plt.axvspan(combined_lower, combined_upper, alpha=0.15)